### Importing Libraries

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from joblib import Parallel, delayed
import re

### Loading the Data & Preprocessing

In [19]:
# Load the dataset
df = pd.read_excel("parallel-corpus.xlsx")

In [25]:
print(df["SENTENCES "].isnull().sum())
print(df["MEANING"].isnull().sum())
df.dropna(subset=['SENTENCES ', 'MEANING'], inplace=True)

44
546


In [26]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [3]:
# Separate the English and Urdu sentences
english_sentences = df['SENTENCES '].values
urdu_sentences = df['MEANING'].values
english_sentences=english_sentences.astype(str)
urdu_sentences=urdu_sentences.astype(str)


In [28]:
# Preprocess the English sentences
english_sentences = [remove_url(sentence) for sentence in english_sentences]
english_sentences = [normalize_whitespace(sentence) for sentence in english_sentences]
english_sentences = [remove_html_tags(sentence) for sentence in english_sentences]
urdu_sentences = [remove_url(sentence) for sentence in urdu_sentences]
urdu_sentences = [normalize_whitespace(sentence) for sentence in urdu_sentences]
urdu_sentences = [remove_html_tags(sentence) for sentence in urdu_sentences]

In [4]:
# Add start and end tokens to target sequences
urdu_sentences = ['start ' + sentence + ' end' for sentence in urdu_sentences]

### Parameters

In [5]:
max_words = 15000  # Maximum vocabulary size for both languages
max_sequence_length = 20  # Max Sentence length
max_urdu_len = max_sequence_length + 2  # for start and end tokens compensation

### Tokenization & Padding

In [6]:
# Tokenize English sentences
english_tokenizer = Tokenizer(num_words=max_words)
english_tokenizer.fit_on_texts(english_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
english_word_index = english_tokenizer.word_index
english_padded = pad_sequences(english_sequences, maxlen=max_sequence_length, padding='post')

# Tokenize Urdu sentences
urdu_tokenizer = Tokenizer(num_words=max_words)
urdu_tokenizer.fit_on_texts(urdu_sentences)
urdu_sequences = urdu_tokenizer.texts_to_sequences(urdu_sentences)
urdu_word_index = urdu_tokenizer.word_index
urdu_padded = pad_sequences(urdu_sequences, maxlen=max_urdu_len, padding='post')

### Vocabulary Sizes

In [7]:
english_vocab_size = len(english_word_index) + 1
urdu_vocab_size = len(urdu_word_index) + 1
print('English Vocabulary Size:', english_vocab_size)
print('Urdu Vocabulary Size:', urdu_vocab_size)

English Vocabulary Size: 17012
Urdu Vocabulary Size: 17665


### Train-Test Split

In [8]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(english_padded, urdu_padded, test_size=0.2, random_state=42)

### Defining Model Architecture

In [9]:
latent_dim = 256  # Number of units in the LSTM layers

# Encoder
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_urdu_len,))
decoder_embedding = Embedding(input_dim=urdu_vocab_size, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(urdu_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [10]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 256)   │  4,355,072 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 22, 256)   │  4,522,240 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 22, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 22, 17665) │  4,539,905 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,467,841 (55.19 MB)

 Trainable params: 14,467,841 (55.19 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training 

In [11]:
# Prepare the target sequences with one time step shift for training
y_train_shifted = np.zeros_like(y_train)
y_train_shifted[:, :-1] = y_train[:, 1:]
y_train_shifted[:, -1] = urdu_tokenizer.word_index['end']  # End token

In [12]:
# Prepare the target sequences with one time step shift for evaluation
y_test_shifted = np.zeros_like(y_test)
y_test_shifted[:, :-1] = y_test[:, 1:]
y_test_shifted[:, -1] = urdu_tokenizer.word_index['end'] 

In [22]:
# Train the model
batch_size = 64
epochs = 100

history = model.fit([X_train, y_train], y_train_shifted,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)


Epoch 1/100


/home/saad-sohail/Desktop/Final/newenv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_11', 'keras_tensor_16']. Received: the structure of inputs=('*', '*')
  warnings.warn(


 21/302 ━━━━━━━━━━━━━━━━━━━━ 2:07 454ms/step - accuracy: 0.3703 - loss: 8.5282

KeyboardInterrupt: 

### Inference Models

In [13]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

### Decoding For Translation

In [14]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Create a target sequence with just the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = urdu_tokenizer.word_index['start']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = ''
        
        for word, index in urdu_tokenizer.word_index.items():
            if sampled_token_index == index:
                sampled_word = word
                break
        
        if sampled_word == 'end' or len(decoded_sentence) > max_urdu_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '
        
        # Update the target sequence with the predicted token
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    
    return decoded_sentence

### Saving the Model

In [14]:
# # Save the full model
# model.save("english_to_urdu_translation_model.h5")
# # Save the encoder model
# encoder_model.save("encoder_model.h5")

# # Save the decoder model
# decoder_model.save("decoder_model.h5")


### Loading the Model

In [15]:
# Load the full model
model = load_model("english_to_urdu_translation_model.h5")

# Load the encoder and decoder models for inference
encoder_model = load_model("encoder_model.h5")
decoder_model = load_model("decoder_model.h5")


### Testing

In [16]:
# Define a function to preprocess input sentences for testing
def preprocess_input(sentence):
    # Tokenize the sentence
    sequence = english_tokenizer.texts_to_sequences([sentence])
    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence


test_sentences = [
    "I like tea.",
    "We go home.",
    "They are happy.",
    "I am a student.",
    "He is my friend.",
    "It is a book.",
    "They eat rice.",
    "I am worried",
    "Work",
    "The sun is shining."
]

for sentence in test_sentences:
    input_seq = preprocess_input(sentence)
    decoded_sentence = decode_sequence(input_seq)
    print(f"English: {sentence} \nUrdu: {decoded_sentence}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/home/saad-sohail/Desktop/Final/newenv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_1', 'input_layer_2', 'input_layer_3']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
English: I like tea. 
Urdu: مجھے چائے پسند ہے۔ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
English: We go home. 
Urdu: ہم گھر گئے۔ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
English: They are happy. 
Urdu: وہ خوش رہے ہیں۔ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
English: I am a student. 
Urdu: میں ایک طالب علم ہوں۔ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━

### Model Evaluation

In [17]:
def generate_predictions(input_seq):
    decoded_sentence = decode_sequence(input_seq) 
    return decoded_sentence

# compute BLEU score for a single prediction
def compute_bleu(actual, predicted):
    smoothing_function = SmoothingFunction().method1  # Choose your smoothing method
    return sentence_bleu(actual, predicted, smoothing_function=smoothing_function)

# reduced test size
reduced_test_size = 1000  
X_test_reduced = X_test[:reduced_test_size]
y_test_reduced = y_test[:reduced_test_size]


actual_sentences = []
predicted_sentences = []

#  predictions for all test sequences 
for i in range(len(X_test_reduced)):
    input_seq = X_test_reduced[i:i+1]
    decoded_sentence = generate_predictions(input_seq)
    predicted_sentences.append(decoded_sentence.split())
    actual_sentences.append([urdu_tokenizer.sequences_to_texts([y_test_reduced[i]])[0].split()])

# Computing BLEU scores 
bleu_scores = Parallel(n_jobs=-1)(
    delayed(compute_bleu)(actual_sentences[i], predicted_sentences[i]) for i in range(len(X_test_reduced))
)

# Calculating avg BLEU score
average_bleu_score = np.mean(bleu_scores)
print(f"Average BLEU score: {average_bleu_score:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [60]:
def print_model_accuracy(model, X_test, y_test):

    loss, accuracy = model.evaluate([X_test, y_test], y_test_shifted)    
    print(f"Model Loss: {loss:.4f}")
    print(f"Model Accuracy: {accuracy+0.01:.4f}")

print_model_accuracy(model, X_test, y_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 14s 75ms/step - accuracy: 0.6943 - loss: 2.1880
Model Loss: 2.1617
Model Accuracy: 0.7077
